# MBTI 조건 탐색

In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'AppleGothic'

pd.options.display.float_format = '{:.2f}'.format
# 특정 모듈의 경고 메시지 무시
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
df = pd.read_csv('final_card.csv')

In [3]:
# 승인건수를 수치형 값으로 변환하는 매핑 사전 생성
mapping = {
    "5번이하": 1,
    "5번초과 10번이하": 6,
    "10번초과 15번이하": 11,
    "15번초과 20번이하": 16,
    "20번 초과": 21
}

# 승인건수 컬럼의 값을 매핑
df["승인건수_수치형"] = df["승인건수"].map(mapping)

# 결과 확인
df

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액,승인건수_수치형
0,202112,4080934272,일반한식,경상북도,경주시,5번이하,20000,1
1,202112,4116127380,일반한식,경상북도,경주시,5번이하,20000,1
2,202112,2914424833,일반한식,경상북도,경주시,5번이하,20000,1
3,202112,2950007638,일반한식,경상북도,경주시,5번이하,60000,1
4,202112,2931907302,일반한식,경상북도,경주시,5번이하,10000,1
...,...,...,...,...,...,...,...,...
188447864,202301,3857466791,일식·회집,경상북도,상주시,5번이하,20000,1
188447865,202301,3767671659,일식·회집,경상북도,상주시,5번이하,40000,1
188447866,202301,4075163762,일식·회집,경상북도,상주시,5번이하,40000,1
188447867,202301,2956984953,일식·회집,경상북도,상주시,5번이하,10000,1


## 5보다 작은 애 삭제

In [4]:
pivot = df.pivot_table(index=['거래년월', '고객ID'], 
                       values='승인건수_수치형', 
                       aggfunc='sum').reset_index()

# 2. 승인건수_수치형 값이 2보다 작은 경우 필터링
to_drop = pivot[pivot['승인건수_수치형'] <= 5][['거래년월', '고객ID']]

# 3. 원래 데이터프레임에서 삭제
df_filtered = df.merge(to_drop, on=['거래년월', '고객ID'], how='left', indicator=True)
df_filtered = df_filtered[df_filtered['_merge'] == 'left_only'].drop('_merge', axis=1)
df_filtered

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액,승인건수_수치형
0,202112,4080934272,일반한식,경상북도,경주시,5번이하,20000,1
1,202112,4116127380,일반한식,경상북도,경주시,5번이하,20000,1
2,202112,2914424833,일반한식,경상북도,경주시,5번이하,20000,1
3,202112,2950007638,일반한식,경상북도,경주시,5번이하,60000,1
4,202112,2931907302,일반한식,경상북도,경주시,5번이하,10000,1
...,...,...,...,...,...,...,...,...
188447864,202301,3857466791,일식·회집,경상북도,상주시,5번이하,20000,1
188447865,202301,3767671659,일식·회집,경상북도,상주시,5번이하,40000,1
188447866,202301,4075163762,일식·회집,경상북도,상주시,5번이하,40000,1
188447867,202301,2956984953,일식·회집,경상북도,상주시,5번이하,10000,1


## E VS I(11204015_result_E_I)

In [99]:
import pandas as pd
import numpy as np

def E_vs_I(df, w, x, y):
    # 거래년월 목록 추출
    unique_months = df['거래년월'].unique()
    
    # 결과를 저장할 빈 리스트 생성
    result_list = []
    
    for month in unique_months:
        # 해당 거래년월의 데이터 필터링
        df_month = df[df['거래년월'] == month]
        
        # 고객ID별 가맹점업종명 고유 개수와 승인건수 합계 계산
        df_group = df_month.groupby('고객ID')['가맹점업종명'].nunique().reset_index()
        a = df_month.groupby('고객ID')['승인건수_수치형'].sum().reset_index()
        df_group['승인건수_총합'] = a['승인건수_수치형']
        df_group['다양성지수'] = df_group['가맹점업종명'] / df_group['승인건수_총합']
        
        # 다양성 분류 초기화
        df_group['다양성분류'] = np.where(df_group['다양성지수'] > w, 'E', 'I')
        
        # 조건에 따른 분류 업데이트
        cond1 = df_group['가맹점업종명'] < x
        df_group.loc[cond1, '다양성분류'] = 'I'
        cond2 = df_group['가맹점업종명'] > y
        df_group.loc[cond2, '다양성분류'] = 'E'
        
        # 결과에 거래년월 추가
        df_group['거래년월'] = month
        
        # 필요한 컬럼만 선택하여 리스트에 추가
        result_list.append(df_group[['거래년월', '고객ID', '다양성분류']])
        
    result_df = pd.concat(result_list, ignore_index=True)
    diversity_counts = result_df['다양성분류'].value_counts()
    per = diversity_counts
    

    
    return [per[0],per[1]]


In [104]:
import numpy as np
import pandas as pd
from tqdm import tqdm


w_values = [ 0.5,0.6,0.7]

x_values = [3,4,5]


y_values = [15,20,25]

# 최적의 값 저장
best_score = +np.inf  # 초기 값 (매우 큰 값으로 설정)
best_w, best_x, best_y = None, None, None

# 결과 저장을 위한 리스트
results = []

# tqdm 적용
total_iterations = len(w_values) * len(x_values) * len(y_values)
with tqdm(total=total_iterations, desc="Grid Search 진행 중") as pbar:
    for w in w_values:
        for x in x_values:
            for y in y_values:
                # Score 계산
                score = 0
                aa = 0
                bb=0
                
                a, b = E_vs_I(df_filtered, w, x, y)
                aa+=a
                bb+=b

                score =abs((aa/(aa+bb) ) -(bb / (aa+bb)))

                # 결과 저장
                results.append({'w': w, 'x': x, 'y': y, 'score': score})

                # 최고값 갱신
                if score < best_score: # 작을 수록 best
                    best_score = score
                    best_w, best_x, best_y = w, x, y

                print(w,x,y,score)
                # tqdm 진행 업데이트
                pbar.update(1)

# 결과를 데이터프레임으로 변환
# results_df = pd.DataFrame(results)

# 결과 출력
print(f"최적의 w: {best_w}, x: {best_x}, y: {best_y}")
print(f"최고 점수: {best_score}")

Grid Search 진행 중:   4%|▎         | 1/27 [00:28<12:15, 28.29s/it]

0.5 3 15 0.1796712160774508


Grid Search 진행 중:   7%|▋         | 2/27 [00:56<11:48, 28.34s/it]

0.5 3 20 0.09283680894750673


Grid Search 진행 중:  11%|█         | 3/27 [01:24<11:17, 28.22s/it]

...

0.7 5 15 0.2940132622100202


Grid Search 진행 중:  96%|█████████▋| 26/27 [12:25<00:29, 29.30s/it]

0.7 5 20 0.42342294257906654


Grid Search 진행 중: 100%|██████████| 27/27 [12:54<00:00, 28.69s/it]

0.7 5 25 0.45455446105704067
최적의 w: 0.5, x: 5, y: 25
최고 점수: 0.02302790562133311


In [5]:
import pandas as pd
import numpy as np

def create_E_I_dataframe(df, w, x, y):
    # 거래년월 목록 추출
    unique_months = df['거래년월'].unique()
    
    # 결과를 저장할 빈 리스트 생성
    result_list = []
    
    for month in unique_months:
        # 해당 거래년월의 데이터 필터링
        df_month = df[df['거래년월'] == month]
        
        # 고객ID별 가맹점업종명 고유 개수와 승인건수 합계 계산
        df_group = df_month.groupby('고객ID')['가맹점업종명'].nunique().reset_index()
        a = df_month.groupby('고객ID')['승인건수_수치형'].sum().reset_index()
        df_group['승인건수_총합'] = a['승인건수_수치형']
        df_group['다양성지수'] = df_group['가맹점업종명'] / df_group['승인건수_총합']
        
        # 다양성 분류 초기화
        df_group['다양성분류'] = np.where(df_group['다양성지수'] > w, 'E', 'I')
        
        # 조건에 따른 분류 업데이트
        cond1 = df_group['가맹점업종명'] < x
        df_group.loc[cond1, '다양성분류'] = 'I'
        cond2 = df_group['가맹점업종명'] > y
        df_group.loc[cond2, '다양성분류'] = 'E'
        
        # 결과에 거래년월 추가
        df_group['거래년월'] = month
        
        # 필요한 컬럼만 선택하여 리스트에 추가
        result_list.append(df_group[['거래년월', '고객ID', '다양성분류']])
    
    # 결과 리스트를 하나의 DataFrame으로 병합
    result_df = pd.concat(result_list, ignore_index=True)
    result_df.rename(columns={'다양성분류': 'E_I'}, inplace=True)
    
    return result_df


In [6]:
result_E_I = create_E_I_dataframe(df_filtered, w=0.5, x=5, y=25)

In [7]:
result_E_I['E_I'].value_counts()

E_I
E    5731010
I    5473005
Name: count, dtype: int64

## N VS S(diversity)

In [8]:
diversity = df_filtered.groupby(['거래년월', '고객ID'])['가맹점_광역시도'].nunique().reset_index()

# 컬럼 이름 변경
diversity.rename(columns={'가맹점_광역시도': '가맹점_광역시도_다양성'}, inplace=True)

# 결과 확인
diversity

,거래년월,고객ID,가맹점_광역시도_다양성
0,202101,3038293200,1
1,202101,3038578153,1
2,202101,3040411889,1
3,202101,3042048047,2
4,202101,3042402348,2
...,...,...,...
11204010,202312,9999569060,5
11204011,202312,9999613037,2
11204012,202312,9999679820,2
11204013,202312,9999704899,1


In [9]:
diversity['S_N'] = diversity['가맹점_광역시도_다양성'].apply(lambda x: 'S' if x >= 3 else 'N')
diversity['S_N'].value_counts()

S_N
N    5648107
S    5555908
Name: count, dtype: int64

## F VS T(df_실용_감성_final)

In [10]:
# 모든 카테고리를 포함한 분류 매핑
category_mapping = {
    '실용': {
        '음식_실용': [
            '정육점', '스넥', '제과점', '기타음료식품', '농·축·수산품', '홍삼제품', 
            '기타건강식품', '인삼제품','위탁급식업'
        ],
        '교통_실용': [
            '택시', '철도', '고속버스', '택시회사', 'SK주유소', 'GS주유소', '쌍용S-OIL', 
            '현대정유(오일뱅크)', 'LPG취급점', '전기차충전소', '유류판매', 'E1가스충전소', 
            'SK가스충전소', '현대정유가스충전소', '주유소', '자동차부품', '가타자동차서비스', 
            '세차장', '자동차정비', '자동차시트·타이어', '윤활유전문판매', '화물운송', 
            '이륜차판매', 'GS가스충전소', '쌍용S-OIL가스충전소'
        ],
        '의료': [
            '종합병원', '치과의원', '치과병원', '한방병원', '동물병원', '의원', '의 원', 
            '건강진단', '약 국', '한약방', '의료용품', '기타의료기관 및 기타의료기기','한 의 원', '병 원(응급실운영)'
        ],
        '쇼핑_실용': [
            '슈퍼마켓', '편의점', '농협하나로클럽', '농,축협직영매장', '화원', 
            '인터넷Mall', 'CATV홈쇼핑', '통신판매업1', '인터넷P/G', '인터넷종합Mall','일반(통신판매)',
            '가전제품', '컴퓨터', '통신기기', '사무용 OA기기', '문구용품', 'PG상품권',
            '주방용구', '주방용식기', '편 의 점','안경','기타유통업', '일반가구', '자사카드발행백화점',
            '미용재료','기타 사무용품','상품권전문판매', '상 품 권', '전자상거래상품권','전자상거래상품권전문판매'
        ],
        '부동산/건축': [
            '부동산중개·임대', '페인트', '조명기구', '목재·석재·철물', '건축용 요업제품', 
            '냉열기기', '보일러·펌프·샷시', '기타건축자재'
        ],
        '교육': [
            '보습학원', '예·체능계학원', '기능학원', '외국어학원', '초중고교육기관', 
            '유아원', '독서실', '학습지교육', '기타 교육기관', '학원'
        ],
        '서비스_실용': [
            '세탁소', '가정용품수리', '주차장', '기타수리서비스', '기타용역서비스',
            '보관및 창고업', '혼례서비스업', '정수기', '가례서비스업', '사무서비스', '종합용역',
            '미용원', '이용원', '기타대인서비스', '조세서비스', '위성방송', 'CATV'
        ],
        '농업/농자재': [
            '농기계', '비료,사료,종자', '농축수산가공품', '미곡상', '기타농업관련'
        ],
        '금융/보험': ['손해보험', '생명보험', '기타보험', '비씨카드 정산용가맹점', '비씨카드 정산용(할인)'],
        '공공서비스': [
            '공공요금대행서비스/소득공제대상', '공공요금대행서비스/소득공제비대상', 
            '구내매점(국가기관등)', '비영리/대상', '비영리/비대상', '통신서비스/소득공제비대상'
        ],
        '통신/IT': ['소프트웨어', '사무·통신기기수리', '이동통신요금'],
        '기계/공구': ['기계공구', '카인테리어', '기타전기제품', 'DP&E']
        #'반려동물': ['애완동물', '수족관']
    },
    
    '감성': {
        '음식_감성': [
            '일반한식', '갈비전문점', '한정식', '중국식', '일식·회집', '서양음식', '주류판매점'
        ],
        '레저/스포츠': [
            '골프경기장', '스크린골프', '볼링장', '수영장', '당구장', '골프연습장', 
            '사우나', '헬스클럽', '안마/스포츠마사지', '스포츠·레져용품', 
            '종합레져타운', '레져업소', '관광여행', '기타레져업소', '노 래 방', '볼 링 장', '수 영 장',
            '골프용품 전문점'
        ],
        '쇼핑_감성': [
            '기념품점', '기타잡화', '정장', '스포츠의류', '아동의류', '신발', '맞춤복점', 
            '화장품','귀금속', '대형할인점', '면 세 점', '침구·수예점',
            '기타의류', '가 방', '신 발', '옷감·직물', '카페트,커튼,천막,지물', '기타가구', '액세서리', 
            '내의판매', '단체복', '연 쇄 점', '캐주얼의류', '제 화'
        ],
        '교통_감성': [
            '여객선', '렌터카', '항공사'
        ],
        '문화': [
            '서점', '영화관', '티켓', '민예·공예품', '화방·표구점', '전문서적', 
            '일반서적', '문화취미기타', '출판 및 인쇄물', '기타서적문구', '완구점', '시 계', '음반영상물'
        ],
        '유흥': [
            '주점', '유흥주점', '단란주점', '칵테일바', '스파'
        ],
        '숙박': [
            '특급호텔', '1급 호텔', '2급 호텔', '여관', '모텔', '콘도', 
            '기타숙박업'
        ],
        '서비스_감성': [
            '피부미용실', '인테리어전문'
        ],
        '반려동물': ['애완동물', '수족관']
    },
    '기타': {
        '기타': [
            '업종미등록', '기타잡화', '기타1', '기타4', '기타 전문점'
        ]
    }
}


In [11]:
upjong_list = [
    '일반한식', '자동차정비', '미용원', '기타레져업소', '기념품점', '약 국', '애완동물', '슈퍼마켓',
       '주점', '유흥주점', '서양음식', 'SK주유소', '정육점', '스넥', '의 원', '농·축·수산품',
       '중국식', '한 의 원', '스포츠·레져용품', '보습학원', '기타음료식품', '쌍용S-OIL', '제과점',
       '통신기기', '화원', '화장품', '기타잡화', '유아원', '기타숙박업', 'GS주유소', '편 의 점',
       'LPG취급점', '철도', '페인트', '출판 및 인쇄물', '정장', '농협하나로클럽', '농,축협직영매장',
       '귀금속', '문구용품', '일식·회집', '주유소', '피부미용실', '티켓', '위탁급식업', '예·체능계학원',
       'SK가스충전소', '비료,사료,종자', '침구·수예점', '자동차부품', '당구장', '가전제품', '인터넷Mall',
       '스포츠의류', '현대정유(오일뱅크)', '가정용품수리', '사무·통신기기수리', '안경', '세탁소', '사우나',
       '일반서적', '노 래 방', '스크린골프', '기계공구', '인터넷P/G', '완구점', '기타의류', '미곡상',
       '기타건강식품', '기타용역서비스', '치과의원', '헬스클럽', '기타수리서비스', '기타건축자재', '일반가구',
       '골프경기장', '병 원(응급실운영)', '자동차시트·타이어', '주방용구', '연 쇄 점', '액세서리',
       '전문서적', '주차장', '특급호텔', '1급 호텔', '치과병원', '민예·공예품', '건축용 요업제품',
       '단란주점', '옷감·직물', '자사카드발행백화점', '인삼제품', '외국어학원', '동물병원', '대형할인점',
       '독서실', '농기계', '조명기구', '세차장', '사무서비스', '컴퓨터', '렌터카', '가 방', 'PG상품권',
       '기타 교육기관', '갈비전문점', '보일러·펌프·샷시', '기능학원', '카페트,커튼,천막,지물', '기타유통업',
       '이륜차판매', '종합레져타운', '카인테리어', '화물운송', '내의판매', '종합병원', '항공사',
       '골프용품 전문점', '목재·석재·철물', '홍삼제품', '의료용품', '기타 사무용품', '미용재료', '기타가구',
       '골프연습장', '신 발', '손해보험', '영화관', '비영리/비대상', '기타대인서비스', '기타전기제품',
       'DP&E', '주방용식기', '가타자동차서비스', '비씨카드 정산용가맹점', '기타농업관련',
       '구내매점(국가기관등)', '아동의류', '이용원', '상 품 권', '조세서비스', '사무용 OA기기',
       '문화취미기타', '인테리어전문', 'GS가스충전소', '쌍용S-OIL가스충전소', '가례서비스업', '시 계',
       '기타의료기관 및 기타의료기기', '비영리/대상', '현대정유가스충전소', '윤활유전문판매', '정수기',
       '공공요금대행서비스/소득공제비대상', '화방·표구점', '볼 링 장', '한약방', '인터넷종합Mall',
       '기타 전문점', '콘도', '통신판매업1', '종합용역', '주류판매점', '2급 호텔', '학습지교육',
       '전기차충전소', '건강진단', '유류판매', '한방병원', '업종미등록', '이동통신요금',
       '통신서비스/소득공제비대상', '기타서적문구', '수족관', 'E1가스충전소', '보관및 창고업', '여객선',
       '택시', 'CATV홈쇼핑', '수 영 장', '농축수산가공품', '소프트웨어', '생명보험', '면 세 점',
       '칵테일바', '관광여행', '기타보험', '초중고교육기관', '비씨카드 정산용(할인)', '고속버스',
       '일반(통신판매)', '레져업소', '기타4', '학원', '기타1', '공공요금대행서비스/소득공제대상',
       '안마/스포츠마사지', '부동산중개·임대', '한정식', '상품권전문판매', '혼례서비스업', '냉열기기',
       '맞춤복점', '단체복', '전자상거래상품권','택시회사', '전자상거래상품권전문판매', '위성방송', 'CATV', '제 화', '캐주얼의류', '음반영상물'
]

In [12]:
import pandas as pd

# 매핑 함수
def map_category(upjong):
    for main_category, subcategories in category_mapping.items():
        for sub_category, keywords in subcategories.items():
            if upjong in keywords:
                return main_category, sub_category
    raise ValueError(f"매핑되지 않은 항목: {upjong}")

# 매핑 실행
mapped_data = []
for upjong in upjong_list:
    try:
        main_category, sub_category = map_category(upjong)
        mapped_data.append([upjong, main_category, sub_category])
    except ValueError as e:
        print(e)

# 결과 저장
df1 = pd.DataFrame(mapped_data, columns=['가맹점업종명', '실용/감성 분류', '세분류'])

# 저장
# df.to_csv('mapped_upjong.csv', index=False, encoding='utf-8-sig')
df1.head()

,가맹점업종명,실용/감성 분류,세분류
0,일반한식,감성,음식_감성
1,자동차정비,실용,교통_실용
2,미용원,실용,서비스_실용
3,기타레져업소,감성,레저/스포츠
4,기념품점,감성,쇼핑_감성


In [13]:
df1['가맹점업종명'].value_counts()

가맹점업종명
일반한식           1
가타자동차서비스       1
기타농업관련         1
구내매점(국가기관등)    1
아동의류           1
              ..
기타건축자재         1
일반가구           1
골프경기장          1
병 원(응급실운영)     1
음반영상물          1
Name: count, Length: 212, dtype: int64

In [14]:
df2 = df1[['가맹점업종명', '실용/감성 분류']]
df_result = pd.merge(df_filtered, df2, on = '가맹점업종명', how = 'left')

In [15]:
df_result.isnull().sum()

거래년월            0
고객ID            0
가맹점업종명          0
가맹점_광역시도        0
가맹점_시군구     92394
승인건수            0
승인금액            0
승인건수_수치형        0
실용/감성 분류        0
dtype: int64

In [ ]:
df_실용_감성_sum = df_result.pivot_table(
    index=['거래년월', '고객ID'],
    columns='실용/감성 분류',
    values='승인금액', 
    aggfunc='sum',
    fill_value=0).reset_index()

df_실용_감성_sum.columns.name = None
df_실용_감성_sum.index.name = None


df_filtered_실용_감성_승인금액 = df_result.pivot_table(
    index=['거래년월', '고객ID'],
    values='승인금액', 
    aggfunc='sum').reset_index()

df_실용_감성_final = pd.merge(df_실용_감성_sum, df_filtered_실용_감성_승인금액, on = ['거래년월', '고객ID'])

In [ ]:
df_실용_감성_final['실용_비율'] = df_실용_감성_final['실용'] / df_실용_감성_final['승인금액']
df_실용_감성_final

,거래년월,고객ID,감성,기타,실용,승인금액,실용_비율
0,202101,3038293200,30000,0,40000,70000,0.57
1,202101,3038578153,50000,0,140000,190000,0.74
2,202101,3040411889,60000,0,100000,160000,0.62
3,202101,3042048047,90000,0,60000,150000,0.40
4,202101,3042402348,80000,0,120000,200000,0.60
...,...,...,...,...,...,...,...
11204010,202312,9999569060,560000,0,1200000,1760000,0.68
11204011,202312,9999613037,20000,0,40000,60000,0.67
11204012,202312,9999679820,60000,0,300000,360000,0.83
11204013,202312,9999704899,50000,0,120000,170000,0.71


In [ ]:
df_실용_감성_final['실용_비율'].describe()

count   11204015.00
mean           0.68
std            0.24
min            0.00
25%            0.52
50%            0.72
75%            0.88
max            1.00
Name: 실용_비율, dtype: float64

In [ ]:
median_ratio = df_실용_감성_final['실용_비율'].median()

# T_F 변수 생성
df_실용_감성_final['T_F'] = df_실용_감성_final['실용_비율'].apply(lambda x: 'T' if x > median_ratio else 'F')
df_실용_감성_final

,거래년월,고객ID,감성,기타,실용,승인금액,실용_비율,T_F
0,202101,3038293200,30000,0,40000,70000,0.57,F
1,202101,3038578153,50000,0,140000,190000,0.74,T
2,202101,3040411889,60000,0,100000,160000,0.62,F
3,202101,3042048047,90000,0,60000,150000,0.40,F
4,202101,3042402348,80000,0,120000,200000,0.60,F
...,...,...,...,...,...,...,...,...
11204010,202312,9999569060,560000,0,1200000,1760000,0.68,F
11204011,202312,9999613037,20000,0,40000,60000,0.67,F
11204012,202312,9999679820,60000,0,300000,360000,0.83,T
11204013,202312,9999704899,50000,0,120000,170000,0.71,F


In [ ]:
df_실용_감성_final['T_F'].value_counts()

T_F
F    5602008
T    5602007
Name: count, dtype: int64

In [ ]:
df_실용_감성_final

,거래년월,고객ID,감성,기타,실용,승인금액,실용_비율,T_F
0,202101,3038293200,30000,0,40000,70000,0.57,F
1,202101,3038578153,50000,0,140000,190000,0.74,T
2,202101,3040411889,60000,0,100000,160000,0.62,F
3,202101,3042048047,90000,0,60000,150000,0.40,F
4,202101,3042402348,80000,0,120000,200000,0.60,F
...,...,...,...,...,...,...,...,...
11204010,202312,9999569060,560000,0,1200000,1760000,0.68,F
11204011,202312,9999613037,20000,0,40000,60000,0.67,F
11204012,202312,9999679820,60000,0,300000,360000,0.83,T
11204013,202312,9999704899,50000,0,120000,170000,0.71,F


## P VS J(10514023_df_grouped)

In [ ]:
# 거래년월별 승인금액 합산
df_grouped = df_filtered.groupby(['고객ID', '거래년월'], as_index=False)['승인금액'].sum()

# 데이터 정렬 (고객ID와 거래년월 기준)
df_grouped = df_grouped.sort_values(by=['고객ID', '거래년월'])

# 직전 달 승인금액 계산 (고객ID별로 shift)
df_grouped['직전달_승인금액'] = df_grouped.groupby('고객ID')['승인금액'].shift(1)

# 변동성 계산
df_grouped['변동성(%)'] = ((df_grouped['승인금액'] - df_grouped['직전달_승인금액']) / df_grouped['직전달_승인금액']) * 100

# 결과 확인
df_grouped = df_grouped.dropna()

# 25% 분위수와 75% 분위수 계산
q25 = df_grouped['변동성(%)'].quantile(0.25)
q75 = df_grouped['변동성(%)'].quantile(0.75)

# 분위수를 기준으로 'P' 또는 'J' 할당
df_grouped['J_P'] = df_grouped['변동성(%)'].apply(lambda x: 'P' if x < q25 or x > q75 else 'J')
df_grouped['J_P'].value_counts()

J_P
J    5257357
P    5256666
Name: count, dtype: int64

In [ ]:
# 거래년월별 승인금액 합산
df_grouped = df_filtered.groupby(['고객ID', '거래년월'], as_index=False)['승인금액'].sum()

# 데이터 정렬 (고객ID와 거래년월 기준)
df_grouped = df_grouped.sort_values(by=['고객ID', '거래년월'])

# 직전 달 승인금액 계산 (고객ID별로 shift)
df_grouped['직전달_승인금액'] = df_grouped.groupby('고객ID')['승인금액'].shift(1)

# 변동성 계산
df_grouped['변동성(%)'] = ((df_grouped['승인금액'] - df_grouped['직전달_승인금액']) / df_grouped['직전달_승인금액']) * 100

In [ ]:
df_grouped

,고객ID,거래년월,승인금액,직전달_승인금액,변동성(%)
0,2004166649,202312,220000,NaN,NaN
1,2004762122,202312,360000,NaN,NaN
2,2006142305,202312,240000,NaN,NaN
3,2006378564,202312,70000,NaN,NaN
4,2008876528,202312,160000,NaN,NaN
...,...,...,...,...,...
11204010,9999998161,202110,220000,50000.00,340.00
11204011,9999998161,202111,360000,220000.00,63.64
11204012,9999998161,202205,100000,360000.00,-72.22
11204013,9999998161,202206,60000,100000.00,-40.00


In [ ]:
df_grouped

In [ ]:
df_grouped = df_grouped[['거래년월', '고객ID' , 'J_P']]

In [ ]:
result_E_I

,거래년월,고객ID,E_I
0,202112,2803619663,E
1,202112,2805865887,I
2,202112,2806047169,E
3,202112,2807044419,I
4,202112,2807135241,E
...,...,...,...
11204010,202301,9999569060,I
11204011,202301,9999679820,I
11204012,202301,9999809845,I
11204013,202301,9999868994,E


In [ ]:
diversity = diversity[['거래년월', '고객ID' , 'S_N']]

In [ ]:
df_실용_감성_final = df_실용_감성_final[['거래년월', '고객ID' , 'T_F']]
df_실용_감성_final

,거래년월,고객ID,T_F
0,202101,3038293200,F
1,202101,3038578153,T
2,202101,3040411889,F
3,202101,3042048047,F
4,202101,3042402348,F
...,...,...,...
11204010,202312,9999569060,F
11204011,202312,9999613037,F
11204012,202312,9999679820,T
11204013,202312,9999704899,F


In [ ]:
result_1 = pd.merge(df_실용_감성_final, diversity, on = ['거래년월', '고객ID'])
result_2 = pd.merge(result_1, result_E_I, on = ['거래년월', '고객ID'])
result_3 = pd.merge(df_grouped, result_2, on = ['거래년월', '고객ID'], how = 'left')

In [ ]:
result_3['MBTI'] = result_3['E_I'] + result_3['S_N'] + result_3['T_F'] + result_3['J_P']

In [ ]:
result_3

,거래년월,고객ID,J_P,T_F,S_N,E_I,MBTI
0,202312,2052239133,J,F,N,E,ENFJ
1,202312,2052346872,P,F,S,I,ISFP
2,202312,2052835078,P,T,S,I,ISTP
3,202312,2053691159,P,T,S,I,ISTP
4,202312,2053744336,J,F,N,I,INFJ
...,...,...,...,...,...,...,...
10514018,202110,9999998161,P,F,N,I,INFP
10514019,202111,9999998161,P,T,N,I,INTP
10514020,202205,9999998161,P,T,N,I,INTP
10514021,202206,9999998161,P,T,N,I,INTP


In [113]:
result_3.to_csv('고객별_MBTI.csv', index = False)

In [111]:
mbti_counts = result_3['MBTI'].value_counts()
mbti_ratios = (mbti_counts / mbti_counts.sum()) * 100
mbti_ratios

MBTI
ISFJ   7.22
ENFP   7.10
ESFP   6.83
ISTJ   6.81
ENTP   6.69
INTJ   6.62
ESTP   6.48
INTP   6.44
ESFJ   6.25
ENFJ   6.07
ESTJ   5.79
INFJ   5.69
ISFP   5.62
ENTJ   5.55
ISTP   5.54
INFP   5.29
Name: count, dtype: float64